In [ ]:
import pandas as pd
from tqdm import tqdm
import ast
tqdm.pandas()
from sqlalchemy import create_engine, URL
from rapidfuzz import fuzz, process, distance
from rapidfuzz.distance import Levenshtein

In [ ]:
df_rel_on_science = pd.read_json('/mnt/hdd01/patentsview/Reliance on Science - Cleantech Patents/df_oaid_cleantech_lang_detect_yake_title_abstract_noun_chunks.json')
df_patstat = pd.read_json('/mnt/hdd01/PATSTAT Working Directory/PATSTAT/df_patstat_cleantech_granted_abstract_metadata.json')

In [ ]:
df_patstat = df_patstat[df_patstat['appln_auth'].apply(lambda x: 'US' in x or 'EP' in x)]

In [ ]:
rel_oaid = df_rel_on_science['id'].tolist()

In [ ]:
patstat_appln_id = df_patstat['appln_id'].tolist()

# Extract Author Information from PATSTAT

In [ ]:
patstat_url_object = URL.create(
    drivername="postgresql+psycopg2",
    username="tie",
    password="TIE%2023!tuhh",
    # host="134.28.58.100",
    host="100.113.100.152",
    port="65432",
    database="PATSTAT_2023",
)
engine = create_engine(patstat_url_object)

In [ ]:
patstat_appln_id_str = ', '.join(map(str, patstat_appln_id))

query = f"""
SELECT pa.appln_id, p.person_id, p.person_name, p.person_name_orig_lg, p.doc_std_name, p.psn_name, p.han_name
FROM tls206_person AS p
INNER JOIN (
    SELECT appln_id, person_id
    FROM tls207_pers_appln
    WHERE CAST(appln_id AS INTEGER) IN ({patstat_appln_id_str})
) AS pa ON p.person_id = pa.person_id
"""
df_patstat_authors = pd.read_sql_query(query, engine)

In [ ]:
df_patstat_authors.to_csv('/mnt/hdd01/Cleantech Network Analysis/df_patstat_authors.csv', index=False)

In [ ]:
df_patstat_authors = pd.read_csv('/mnt/hdd01/Cleantech Network Analysis/df_patstat_authors.csv')

# Extract Author Information from OpenAlex

In [ ]:
oa_url_object = URL.create(
    drivername="postgresql+psycopg2",
    username="tie",
    password="TIE%2023!tuhh",
    # host="134.28.58.100",
    host="100.113.100.152",
    port="45432",
    database="openalex_db",
)
engine = create_engine(oa_url_object)

In [ ]:
df_oa_authors = pd.DataFrame()

chunk_size = 10000
rel_oaid_chunks = [rel_oaid[i:i + chunk_size] for i in range(0, len(rel_oaid), chunk_size)]

for rel_oaid_chunk in tqdm(rel_oaid_chunks):
    rel_oaid_str = ', '.join(map(repr, rel_oaid_chunk))
    query = f"""
    SELECT wa.work_id, a.id, a.display_name, a.display_name_alternatives
    FROM openalex.authors AS a
    INNER JOIN (
        SELECT work_id, author_id
        FROM openalex.works_authorships
        WHERE work_id IN ({rel_oaid_str})
    ) AS wa ON a.id = wa.author_id
    """
    df_chunk = pd.read_sql_query(query, engine)
    df_oa_authors = pd.concat([df_oa_authors, df_chunk])

df_oa_authors.reset_index(drop=True, inplace=True)

In [ ]:
df_oa_authors.to_csv("/mnt/hdd01/Cleantech Network Analysis/df_oa_authors.csv", index=False)

In [ ]:
df_oa_authors = pd.read_csv("/mnt/hdd01/Cleantech Network Analysis/df_oa_authors.csv")

# Merge Authors from PATSTAT and OpenAlex

In [ ]:
# df_oa_authors['display_name_alternatives'] = df_oa_authors['display_name_alternatives'].apply(lambda x: ast.literal_eval(x)) # already a list
df_oa_authors['display_name_alternatives'] = df_oa_authors.apply(lambda row: row['display_name_alternatives'] + [row['display_name']] if isinstance(row['display_name_alternatives'], list) else [row['display_name']], axis=1)
df_oa_authors_exploded = df_oa_authors.explode('display_name_alternatives')
df_oa_authors_exploded['oaid'] = df_oa_authors_exploded['work_id'].apply(lambda x: x.replace("https://openalex.org/W", ""))
df_oa_authors_exploded['display_name_alternatives'].dropna(inplace=True)
df_oa_authors_exploded['display_name_alternatives'] = df_oa_authors_exploded['display_name_alternatives'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df_oa_authors_exploded.dropna(subset=['display_name_alternatives'], inplace=True)

In [ ]:
df_patstat_authors =df_patstat_authors.drop_duplicates(subset=['appln_id', 'person_id'])
df_patstat_authors_melted = df_patstat_authors.melt(id_vars=['appln_id', 'person_id'], 
                                                                  value_vars=['person_name', 'person_name_orig_lg', 'doc_std_name', 'psn_name', 'han_name'],
                                                                  var_name='name_type', 
                                                                  value_name='name')
df_patstat_authors_melted['name'] = df_patstat_authors_melted['name'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df_patstat_authors_melted['appln_id'] = df_patstat_authors_melted['appln_id'].astype(str)
df_patstat['appln_id'] = df_patstat['appln_id'].astype(str)
df_patstat_authors_melted = pd.merge(df_patstat_authors_melted, df_patstat[['appln_id']], on='appln_id', how='inner')

In [ ]:
df_rel_pcs = pd.read_csv("/mnt/hdd01/Reliance on Science/Raw Files/_pcs_oa.csv") # Only merge authors if patent-paper citation is present

In [ ]:
df_rel_pcs['patent_id_prefix'] = df_rel_pcs['patent'].apply(lambda x: x.split('-')[0])
df_rel_pcs['patent_id'] = df_rel_pcs['patent'].apply(lambda x: '-'.join(x.split('-')[1:-1]))
df_rel_pcs = df_rel_pcs[df_rel_pcs['patent_id_prefix'].isin(['us', 'ep'])]
df_rel_pcs = df_rel_pcs[df_rel_pcs['oaid'].isin(df_rel_on_science['oaid'])]

## Extract Appln_id from PATSTAT

In [ ]:
patstat_url_object = URL.create(
    drivername="postgresql+psycopg2",
    username="tie",
    password="TIE%2023!tuhh",
    # host="134.28.58.100",
    host="100.113.100.152",
    port="65432",
    database="PATSTAT_2023",
)
engine = create_engine(patstat_url_object)

In [ ]:
df_rel_pcs['patent_id_prefix'] = df_rel_pcs['patent_id_prefix'].apply(lambda x: x.upper())
df_rel_pcs.to_sql('temp_df_rel_pcs', engine, if_exists='replace', index=False)

In [ ]:
query = """
SELECT temp.patent_id, temp.patent_id_prefix, publn.appln_id
FROM temp_df_rel_pcs AS temp
INNER JOIN tls211_pat_publn AS publn
ON temp.patent_id = publn.publn_nr AND temp.patent_id_prefix = publn.publn_auth
"""
df_result = pd.read_sql_query(query, engine)

In [ ]:
df_result['patent_id_prefix_patent_id'] = df_result['patent_id_prefix'] + '-' + df_result['patent_id']
df_rel_pcs['patent_id_prefix_patent_id'] = df_rel_pcs['patent_id_prefix'] + '-' + df_rel_pcs['patent_id']
df_rel_pcs.drop(columns=['patent', 'wherefound', 'confscore', 'self', 'reftype'], inplace=True)

In [ ]:
df_result = df_result[['patent_id_prefix_patent_id', 'appln_id']]
df_rel_pcs = df_rel_pcs[['oaid', 'patent_id_prefix_patent_id']]

df_result.set_index('patent_id_prefix_patent_id', inplace=True)
df_rel_pcs.set_index('patent_id_prefix_patent_id', inplace=True)

df_rel_pcs = df_rel_pcs.join(df_result, how='left')

df_rel_pcs.reset_index(inplace=True)
df_rel_pcs.drop_duplicates(subset=['oaid', 'appln_id'], inplace=True)

In [ ]:
df_rel_pcs_grouped = df_rel_pcs.groupby('appln_id').agg({'oaid': list}).reset_index()

## Continue

In [ ]:
df_patstat_authors_melted['appln_id'] = df_patstat_authors_melted['appln_id'].astype(str)
df_rel_pcs_grouped['appln_id'] = df_rel_pcs_grouped['appln_id'].astype(str)
df_patstat_authors_melted = pd.merge(df_patstat_authors_melted, df_rel_pcs_grouped[['oaid', 'appln_id']], left_on='appln_id', right_on='appln_id', how='inner', validate='m:m')
# df_patstat_person_details_melted = df_patstat_person_details_melted.dropna(subset=['oaid'])
df_patstat_authors_melted = df_patstat_authors_melted.dropna(subset=['appln_id'])
df_patstat_authors_exploded = df_patstat_authors_melted.explode('oaid')
df_patstat_authors_exploded = df_patstat_authors_exploded.dropna(subset=['oaid'])

In [ ]:
df_oa_authors_exploded_exploded = df_oa_authors_exploded.explode('oaid')

In [ ]:
df_patstat_authors_exploded.head()

In [ ]:
df_oa_authors_exploded_exploded.rename(columns={'id': 'author_id'}, inplace=True)
df_oa_authors_exploded_exploded['oaid'] = df_oa_authors_exploded_exploded['oaid'].astype(str)
df_oa_authors_exploded_exploded.set_index('oaid', inplace=True)

df_patstat_authors_exploded['oaid'] = df_patstat_authors_exploded['oaid'].astype(str)
df_patstat_authors_exploded.set_index('oaid', inplace=True)

df_merged = df_oa_authors_exploded_exploded.join(df_patstat_authors_exploded, how='inner')

df_merged = df_merged[['appln_id', 'person_id', 'name_type', 'name', 'author_id', 'display_name', 'display_name_alternatives']]
df_merged.reset_index(inplace=True)

In [ ]:
def match_names(row):
    full_name = row['name']
    match = distance.Levenshtein.normalized_similarity(full_name, row['display_name_alternatives'])
    return match

df_merged['best_match'] = df_merged.progress_apply(match_names, axis=1)
# df_merged_test['best_match'] = df_merged_test.progress_apply(match_names, axis=1)

# df_merged = df_merged.sort_values('best_match', ascending=False)
df_merged_filtered = df_merged[df_merged['best_match'] >= 0.75]
df_merged_filtered = df_merged_filtered.loc[df_merged_filtered.groupby(['appln_id', 'person_id', 'oaid', 'author_id', 'display_name'])['best_match'].idxmax()]

In [ ]:
df_patstat_authors_filtered = df_patstat_authors[~df_patstat_authors[['appln_id', 'person_id']].apply(tuple, 1).isin(df_merged_filtered[['appln_id', 'person_id']].apply(tuple, 1))]
df_patstat_authors_filtered = df_patstat_authors_filtered.drop_duplicates(subset=['appln_id', 'person_id'])

In [ ]:
df_patstat_authors['appln_id'] = df_patstat_authors_filtered['appln_id'].astype(str)
df_patstat['appln_id'] = df_patstat['appln_id'].astype(str)
df_patstat_authors_filtered['appln_id'] = df_patstat_authors_filtered['appln_id'].astype(str)
df_patstat['appln_id'] = df_patstat['appln_id'].astype(str)
df_patstat_authors_filtered = pd.merge(df_patstat_authors_filtered, df_patstat, on='appln_id', how='inner', validate='m:m')
# df_patstat_authors_filtered = df_patstat_authors_filtered.rename(columns={'publn_nr': 'patent_id'})

In [ ]:
# df_oa_authors_exploded_exploded = df_oa_authors_exploded_exploded.reset_index()
df_oa_authors_filtered = df_oa_authors_exploded_exploded[~df_oa_authors_exploded_exploded[['oaid', 'author_id']].apply(tuple, 1).isin(df_merged_filtered[['oaid', 'author_id']].apply(tuple, 1))]
df_oa_authors_filtered = df_oa_authors_filtered.drop_duplicates(subset=['oaid', 'author_id'])
df_oa_authors_filtered = df_oa_authors_filtered[['oaid', 'author_id', 'display_name', 'display_name_alternatives']]

In [ ]:
df_authors = pd.concat([df_merged_filtered, df_patstat_authors_filtered, df_oa_authors_filtered], ignore_index=True)

In [ ]:
df_authors.drop_duplicates(subset=['appln_id', 'person_id', 'oaid', 'author_id'], inplace=True)

In [ ]:
df_authors.to_csv('/mnt/hdd01/Cleantech Network Analysis/df_authors.csv', index=False)